# 05. MEMORY

langchain에는 5가지 종류의 메모리가 있다. 각자 저장 방법도 다르고, 장단점도 다르다.   
챗봇에 메모리를 추가하지 않으면 아무것도 기억할 수 없기 때문에, 이어지는 질문을 해도 기억된 답변이 나올 수 없음. (stateless)   
대화형 챗봇을 만들기 위해서는 대화 느낌을 줄 수 있는 MEMORY 사용이 필수적이다.   
모델 자체에는 메모리가 없기 떄문에, 모델에게 요청을 보낼 땐 이전 대화 기록을 보내줘야 함.   

메모리를 다루는 측면에서는 최대한 적은 대화 내용을 저장해 이후 답변 내용에 최대한 반영하는 것이 목표가 될 것임.

### 1. ConversationBufferMemory
- 단순히 이전 대화 내용 전체를 저장한다.
- 대화가 길어질 수록 메모리 효율이 떨어짐

### 2. ConversationBufferWindowMemory
- ConverationBufferMemory가 모든 대화내용을 저장함에 따라 발생하는 메모리 비효율을 일부 완화
- 저장량에 Limit을 두어, 모든 대화가 아닌 최근 대화만을 저장한다는 차이가 있음.
- 즉, 메모리의 임계치를 넘지 않을 수 있다는 장점을 가짐,
- 단 버퍼의 크기를 넘어 lost된 대화 내용은 기억할 수 없다는 단점이 있음. 

### 3. ConversationSummaryMemory (ChatOpenAI)
- message를 그대로 저장하는 것이 아닌, conversation의 요약을 자체적으로 해 줌.
- 매우 긴 Conversation이 있는 경우 유리함. 
- 초반에는 더 많은 토큰과 저장 공간이 필요해 짧은 대화에는 비효율적일 수 있음.

### 4. ConversationSummaryBufferMemory (★)
- ConversationSummary + ConversationBuffer
- 메모리에 보내 온 메시지를 그대로 저장하며 그 수를 셈
- 지정한 limit에 다다른 순간 ***오래된 메시지를 Summarize***함.

### 5. ConversationKGMemory (KG : Knowledge Graph)
- 대화 중 Entity의 KG를 만들어 가장 중요한 것들(로 판단되는 것들)을 요약
- history가 아닌 ***Entity를 가지고 오기 때문에*** 예제의 get_history method는 사용하지 않았음.


In [1]:


import os
os.environ["TIKTOKEN_CACHE_DIR"]="./etc"


from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

## FewShotPromptTemplate 실습을 위해 import
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate

## ConversationBuffer(Window)Memory
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
## ConversationSummaryMemory
from langchain.memory import ConversationSummaryMemory, ConversationSummaryBufferMemory
## ConversationKGMemory
from langchain.memory import ConversationKGMemory

## LCEL Based Memory
from langchain.schema.runnable import RunnablePassthrough

from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,     ##Streaming 옵션 ON
    callbacks=[StreamingStdOutCallbackHandler()]
)



In [2]:
## memory = ConversationBufferMemory()
## memory = ConversationBufferMemory(return_messages=True)
## memory = ConversationSummaryMemory(llm=chat)

"""
memory = ConversationKGMemory(llm=chat, return_messages=True)


def add_message(input, output):
    memory.save_context({"input":input,}, {"output":output})

def get_history():
    memory.load_memory_variables({})
"""

## memory에 현재까지의 대화 context를 저장 (수동)
##memory.save_context({"input":"Hi!"}, {"output":"How are you?"})

##memory.load_memory_variables({})

## output ##
## {'history': 'Human: Hi!\nAI: How are you?'}
## {'history': [HumanMessage(content='Hi!'), AIMessage(content='How are you?')]} -- return_messages=true

memory = ConversationSummaryBufferMemory(llm=chat, return_messages=True)
memory.save_context({"input":"Hi!"}, {"output":"How are you?"})


### ConversationBuffer의 결과.
save_context를 통해 넣어 주었던 대화 내용이 string으로 출력된다.
챗 모델을 사용하기 위해서는 AI/Human의 데이터가 각각 필요하지만 현재의 output 은 단순 string이므로,   
챗봇 목적으로 메모리를 사용할 경우 처음 memory를 생성할 때 return_messages=True로 설정한다.      
이 경우 HumanMessage, AIMessage가 분리되어 list형태로 반환한다.   
이 과정을 반복하면 동일한 대화 내역이 계속 저장되어 메모리 비효율 발생   

### 참고
모든 메모리는 save_context, load_memory_variables 라는 함수를 가지고 있다.   
즉 메모리의 종류만 결정한다면 아래 단의 코드 수정은 필요 없음.

In [3]:
## ConversationSummaryMemory
"""
add_message("Hi, I'm Nicolas, I live in South Korea", "Wow that is so cool!")
add_message("Nicolas likes kimchi", "Wow that is so cool!")
##add_message("South Korea is so pretty", "I wish I could go!!!")

##get_history()

## KG memory load
memory.load_memory_variables({"input":"who is Nicolas"})

## output (ConversationSummaryMemory)
## The human greets the AI with a "Hi!" and the AI responds by asking how the human is. 
## The human introduces themselves as Nicolas from South Korea, and the AI responds by saying that it is cool. 
## The human mentions that South Korea is so pretty, and the AI expresses a wish to go there.

## output (ConversationKGMemory)
## (Nicolas, lives in, South Korea)Nicolas
## (Nicolas, lives in, South Korea)(Nicolas, likes, kimchi)Nicolas
"""



'\nadd_message("Hi, I\'m Nicolas, I live in South Korea", "Wow that is so cool!")\nadd_message("Nicolas likes kimchi", "Wow that is so cool!")\n##add_message("South Korea is so pretty", "I wish I could go!!!")\n\n##get_history()\n\n## KG memory load\nmemory.load_memory_variables({"input":"who is Nicolas"})\n\n## output (ConversationSummaryMemory)\n## The human greets the AI with a "Hi!" and the AI responds by asking how the human is. \n## The human introduces themselves as Nicolas from South Korea, and the AI responds by saying that it is cool. \n## The human mentions that South Korea is so pretty, and the AI expresses a wish to go there.\n\n## output (ConversationKGMemory)\n## (Nicolas, lives in, South Korea)Nicolas\n## (Nicolas, lives in, South Korea)(Nicolas, likes, kimchi)Nicolas\n'

### ConversationSummary의 결과
대화 내용 자체를 저장하지 않고, ***대화에서 벌어지고 있는 상황에 대한 요약글***을 자체 제작해 저장함.   
위 예제의 경우 add한 데이터의 양보다 Summary의 양이 더욱 많으나, 대화량이 많아질수록 Summarized text의 효율이 증가할 것임.


* Issue
HTTPSConnectionPool(host='openaipublic.blob.core.windows.net', port=443): Max retries exceeded with url 에러 발생   
인증서 확인 과정에서 문제가 발생한 것으로 보임   
1. requests, certifi 업그레이드 => 해결 안됨
2. requests, certifi 통해 url 검증 강제 False => 해결 안됨
=> CA pem 파일의 문제인 것으로 보임, 실행하지 못했음

★ tiktoken 사용 시 네트워크 환경으로 인한 SSL 오류로,   
필요한 파일을 다운받아 로컬 캐시로 지정함

(코드 맨 위의 os.verion["TIKTOKEN_CACHE_DIR"]="./env")

### ConversationKGMemory의 결과
대화 내의 Entity 기반으로 내용을 저장함. 예제의 경우 'Nicolas' 라는 Entity에 대한 정보를 저장하였음.   
* 그럼 이 정보들을 하나하나 다 입력해 주어야 하는가? (save_context, load_variables의 반복)   

## Memory on LLMChain
- 메모리를 chain에 연결하는 방법과 두 종류의 chain을 사용하는 방법에 대해 설명   
- LLMChain : off-the-shelf chain (일반적인 목적을 가진 chain)

In [4]:
from langchain.chains import LLMChain

##ConversationSummaryBufferMemory 오류로 BufferMemory 사용
"""
memory = ConversationBufferMemory(
    llm=chat,
    ##max_token_limit=80,
    memory_key="chat_history",
    return_messages=True,   ##문자열이 아닌 Message로 반환
)
"""

memory = ConversationSummaryBufferMemory(
    llm=chat,
    memory_key="chat_history",
    return_messages=True,
)

## PromptTemplate 사용 시
template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""

## ChatPromptTemplate 사용 시
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human"),
    ## 대화 저장 memory의 내용으로 MessagePlaceholder를 채움
    ## Memory가 Sysetm/Human/AI Message를 주면 얼마나 올지 모르기에 placeholder에 담음
    MessagesPlaceholder(variable_name="chat_history"),      ## 누구의, 얼마나 큰 메시지인지 알 수 없는 공간
    ("human", "{question}"),
])

chain = LLMChain(
    llm=chat,
    memory=memory,
    prompt=prompt,
    verbose=True,   ## Chain의 Prompt log를 확인 가능
)
## LLMChain의 경우 위의 format으로만 구성이 가능함


In [5]:
chain.predict(question="My name is Nico")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Nico
Nice to meet you, Nico! How can I assist you today?
> Finished chain.


'Nice to meet you, Nico! How can I assist you today?'

In [12]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Nico
AI: Hello Nico! How can I assist you today?
Human: I live in Seoul
Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?
> Finished chain.


'Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?'

In [13]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Nico
AI: Hello Nico! How can I assist you today?
Human: I live in Seoul
AI: Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?
Human: What is my name?
Your name is Nico.
> Finished chain.


'Your name is Nico.'

각 독립적인 질문이 후속 질문에 영향을 주지 못하고 있음.   
우리가 Prompt에게 대화 내역을 말해준 적이 없기 떄문임. (Memory에게 History를 넘기는 것과는 별개임.)   
(My name is Nico -> What is my name?)   

Memory에는 대화 기록들이 정상적으로 저장되고 있지만, 이 데이터를 Prompt에게 넘겨주지를 못하는 상황   
따라서 우리가 원하는 어떠한 방식으로 Prompt에게 대화 기록을 추가해 주어야 함   

In [8]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='My name is Nico'),
  AIMessage(content='Nice to meet you, Nico! How can I assist you today?'),
  HumanMessage(content='I live in Seoul'),
  AIMessage(content='Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or talk about regarding Seoul?'),
  HumanMessage(content='What is my name?'),
  AIMessage(content='Your name is Nico. How can I assist you further, Nico?')]}

## LCEL Based Memory (LCEL : LangChain Expression Language)
생성된 chain에 메모리를 추가하는 것은 어렵지 않음   

* RunnablePassthrough
- chain에서 prompt의 앞 순서에 실행되어 그 결과를 prompt에게 넘겨줌
- prompt의 format 이전에 함수를 실행시킬 수 있어 변수 세팅에 용이함

In [9]:
def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]

"""
chain = prompt | chat
chain.invoke({
    "chat_history":memory.load_memory_variables({})["chat_history"],
    "question": "My name is Nico"
})

"""

## 이 코드는 위와 동일한 결과를 낸다.
chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | chat
chain.invoke({
    "question": "My name is Nico"
})



Yes, you mentioned that your name is Nico. How can I assist you today, Nico?

AIMessageChunk(content='Yes, you mentioned that your name is Nico. How can I assist you today, Nico?')

다음으로는 각 결과를 메모리에 수동으로 저장하고 있는 점을 해결할 필요가 있음.

In [10]:
def invoke_chain(question):
    result = chain.invoke({
        "question": "My name is Nico"
    })
    memory.save_context({"input":question}, {"outpt":result.content})
    ##print(result)

In [11]:
invoke_chain("My name is Nico")

Yes, I understand that your name is Nico. How can I assist you today, Nico?content='Yes, I understand that your name is Nico. How can I assist you today, Nico?'


In [12]:
invoke_chain("What is my name?")

Got it, Nico! How can I assist you today?content='Got it, Nico! How can I assist you today?'


참고 : memory_key는 default값이 'history'이다.